In [18]:
import os
import re
import string
import annoy

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
nltk.download('stopwords')

from pymorphy2 import MorphAnalyzer
from string import punctuation
from gensim.models import Word2Vec

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

[nltk_data] Downloading package stopwords to /home/nlp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [79]:
df = pd.read_csv('ProductsDataset.csv')

In [80]:
df.head()

,title,description,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


### Text Preprocessing

In [172]:
def preprocess_text(txt: str, sw = []) -> str:
    exclude = list(punctuation)
    '''Стоп-слова из библиотеки nltk'''
    sw_1 = list(stopwords.words('russian'))
    morpher = MorphAnalyzer()
    txt = str(txt)
    '''Делим предложение на слова по знакам препинания, приводим все слова к нижнему регистру, удаляем знаки препинания, стоп-слова и цифры'''
    tokenizer = RegexpTokenizer("\w+|[^\w\s]+")
    txt = [i.lower() for i in tokenizer.tokenize(txt) if i not in exclude and i not in sw and i not in sw_1 and len(i) > 2 and \
          re.search('(\d+)|\)|\(|\.|\:|\;|\!|\?|\<|\>|[A-Za-z]', i) is None]
    '''Делаем лемматизацию'''
    txt = [morpher.parse(word)[0].normal_form for word in txt]
    return " ".join(txt)

In [81]:
df.rename(columns = {'descrirption' : 'description'}, inplace = True) 
df = df.fillna('0')
df['description_full'] = df['title'] + ' ' + df['description']
df = df.loc[df['description_full'] != '00']

In [82]:
df

,title,description,product_id,category_id,subcategory_id,properties,image_links,description_full
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"Юбка детская ORBY Новая, не носили ни разу. В ..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"Ботильоны Новые,привезены из Чехии ,указан раз..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,Брюки Размер 40-42. Брюки почти новые - не зна...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"Продам детские шапки Продам шапки,кажда 200р.Р..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"Блузка Темно-синяя, 42 размер,состояние отличн..."
...,...,...,...,...,...,...,...,...
35543,Юбка,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,9.0,904,"{'zhenskaya_odezhda_platya_yubki_tip': 'Юбки',...",http://cache3.youla.io/files/images/360_360/5b...,Юбка Юбка Белая по.Турция фирма adL
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0,9.0,908,"{'brand_zhenskii': 'Chanel', 'zhenskaya_odezhd...",http://cache3.youla.io/files/images/360_360/5b...,Новый твидовый пиджак Новый с бирками пиджак р...
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2,9.0,903,"{'zhenskaya_odezhda_razmer': '48-50 (XL)', 'zh...",http://cache3.youla.io/files/images/360_360/5b...,Женская зимняя куртка Женская зимняя спортивна...
35546,Новая золотая ветровка,Женская ветровка размер 44-46. Цвет приглушённ...,5bd6c8fb2138bbc55745362c,9.0,903,"{'zhenskaya_odezhda_razmer': '44-46 (М)', 'zhe...",http://cache3.youla.io/files/images/360_360/5b...,Новая золотая ветровка Женская ветровка размер...


In [190]:
df['description_full_norm'] = df.apply(lambda x: preprocess_text(x['description_full'], sw), axis=1)

In [34]:
df_product = pd.DataFrame()
df_product['description'] = df['description_full_norm']
df_product['target'] = 1

In [88]:
df_product = df_product.loc[df_product['description'] != 'nan']
df_product

,description,target
0,юбка детский orby новый носить раз в реал крас...,1
1,ботильон новый привезти чехия указать размер м...,1
2,брюки размер брюки новый знать мерило покупка ...,1
3,продать детский шапка продать шапка кажда розо...,1
4,блузка темно синий размер состояние отличный н...,1
...,...,...
35542,новый болеро s oliver привезти европа подойти ...,1
35543,юбка юбка белый турция фирма adl,1
35544,новый твидовый пиджак новый бирка пиджак разме...,1
35545,женский зимний куртка женский зимний спортивны...,1


## 2

In [89]:
question = None
written = False

'''Идем по всем записям, берем первую строку как вопрос и после знака --- находим ответ'''
with open('prepared_answers.txt', 'w') as fout:
    with open('answers.txt', 'r') as fin:
        for line in tqdm_notebook(fin):
            if line.startswith('---'):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace('\t', ' ').strip() + '\t' + line.replace('\t', ' '))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/tmp/ipykernel_635881/1279344030.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [94]:
import tqdm

In [97]:
prepared_sentences = []
c = 0

with open('prepared_answers.txt', 'r') as fin:
    for line in tqdm.notebook.tqdm(fin):
        spls = preprocess_text(line, sw)
        prepared_sentences.append(spls)
        c += 1
        if c >= len(df_product):
            break

0it [00:00, ?it/s]

In [98]:
df_chat = pd.DataFrame()
df_chat['description'] = prepared_sentences
df_chat['target'] = 0

In [101]:
main_df = pd.concat([df_product, df_chat], ignore_index=True)

In [105]:
main_df

,description,target
0,юбка детский orby новый носить раз в реал крас...,1
1,ботильон новый привезти чехия указать размер м...,1
2,брюки размер брюки новый знать мерило покупка ...,1
3,продать детский шапка продать шапка кажда розо...,1
4,блузка темно синий размер состояние отличный н...,1
...,...,...
67069,гарип асгат гарипович p асга ́ так гарипович г...,0
67070,фото там >>>>>>>>> а сказать такой мужчина ?? ...,0
67071,избавиться монобровь навсегда хотеть убрать мо...,0
67072,citroen jumper неисправность кпп citroen jumpe...,0


## 3

In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
import numpy as np

In [107]:
vectorizer = TfidfVectorizer()

In [108]:
X = main_df['description']
y = main_df['target']

In [109]:
X = vectorizer.fit_transform(X)
y = np.array(y)

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [117]:
X_train, y_train

(<33537x132368 sparse matrix of type '<class 'numpy.float64'>'
 	with 771378 stored elements in Compressed Sparse Row format>,
 array([1, 1, 1, ..., 0, 0, 1]))

In [164]:
from joblib import dump, load
import pickle

## 4

In [120]:
log_reg = LogisticRegression(random_state=0).fit(X_train, y_train)

In [157]:
y_pred = log_reg.predict(X_test)

print(type(predictions))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [158]:
accuracy = (y_pred == y_test).mean()
accuracy

0.9857470853087634

In [169]:
def save_model(model, name):
    return pickle.dump(model, open(f'models/{name}.sav', 'wb'))

In [170]:
save_model(log_reg, 'logreg_model')

In [171]:
pickle.load(open('models/logreg_model.sav', 'rb'))

LogisticRegression(random_state=0)

In [175]:
positive_comment = preprocess_text('толстовка', sw)

vec = vectorizer.transform([positive_comment])
log_reg.predict(vec)

array([1])

In [160]:
negative_comment = 'День Рождения'

vec = vectorizer.transform([negative_comment])
log_reg.predict(vec)

array([0])

In [161]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train,y_train)

In [ ]:
y_pred = svm.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))

## 5

In [191]:
product_list = df['description_full_norm'].tolist()

In [193]:
model = Word2Vec(sentences=product_list, vector_size=100, min_count=1, window=5)

In [195]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open('prepared_answers.txt', 'r') as f:
    for line in tqdm.notebook.tqdm(f):
        n_w2v = 0
        spls = line.split('\t')
        index_map[counter] = spls[1]
        question = preprocess_text(spls[0], sw)
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)

index.build(10)
#index.save('speaker.ann')

0it [00:00, ?it/s]

## 6

In [173]:
def get_answer():
    return 0

In [ ]:
text = 'толстовка'

In [176]:
log_reg = pickle.load(open('models/logreg_model.sav', 'rb'))

In [187]:
text_norm = preprocess_text('толстовка', sw)
vec = vectorizer.transform([text_norm])
predict_class = log_reg.predict(vec)

In [186]:
if predict_class[0] == 0:
    find_answer(text_norm)

1

### Text Vectorization

In [17]:
sw = ['всё', 'тому', 'представьте', 'чьё', 'смысле', 'статься', 'знаешь', 'получается', 'или', 'выяснилось', 'удивительно',
      'которых', 'оти', 'значится', 'сказано', 'никогда', 'об', 'образом', 'моему', 'самом', 'свыше', 'во-вторых', 'как', 'чья', 'иметь', 
      'следовательно', 'серьезно', 'точки', 'точнее', 'поди', 'бы', 'но', 'само', 'осле', 'конено', 'около', 'из-за', 'отому', 'скажем',
      'новость', 'достаточно', 'издревле', 'когда', 'любом', 'ваша', 'мол', 'конец', 'словно', 'никто', 'уж', 'да', 'актуальный', 'попросту',
      'бесспорно', 'долженствовать', 'было', 'которая', 'хорошо', 'случайно', 'хоть', 'ко', 'видать', 'повседневной', 'этот', 'позволь',
      'наверно', 'прочему', 'самым', 'буквально', 'здесь' 'воистину', 'право', 'ничего', 'понятно', 'разнообразный', 'вообразите', 'июль',
      'эта', 'тогда', 'дальше', 'конечно', 'наоборот', 'бывает', 'твоя', 'мне', 'полагать', 'важно', 'нам', 'представленный', 'проще',
      'очередной', 'выясняется', 'странно', 'устоявшееся', 'моя', 'всевозможный', 'долженствующий', 'прочий', 'где', 'откровенно', 'принято',
      'беспрецедентный', 'целом', 'один', 'нашей', 'абсолютно', 'следствие', 'также', 'они', 'после', 'согласиться', 'реальный', 'по-твоему',
      'имеется', 'лет', 'небось', 'казалось', 'своему', 'скорее', 'неизгладимый', 'напротив', 'март', 'наверняка', 'обычно', 'нынче',
      'моей', 'примечательно', 'против', 'наконец', 'так', 'тыс', 'какой', 'много', 'ней', 'если', 'должно', 'ваш', 'нее', 'быть', 'этом',
      'чем', 'возможно', 'вестимо', 'знать', 'вроем', 'выражаясь', 'одним', 'теерь', 'кажется', 'многие', 'от', 'есть', 'себе', 'просто',
      'всяком', 'даже', 'ничто', 'обычаю', 'знает', 'исходя', 'несколько', 'предельно', 'ли', 'она', 'ерез', 'вас', 'ведь', 'имеются', 'более',
      'он', 'кто', 'по', 'уть', 'свою' 'твоему', 'них', 'кстати', 'од', 'крайне', 'ри', 'говори', 'примеру', 'их', 'вашему', 'всякого',
      'исключение', 'через', 'того', 'год', 'оказывается', 'по-ихнему', 'вишь', 'конце', 'вашего', 'тоб', 'кб', 'пожалуйста', 'сейас',
      'вам', 'эй', 'над', 'естественно', 'без', 'нибудь', 'нечего', 'еред', 'куда', 'вообще-то', 'менее', 'по-вашему', 'хм', 'что', 'про',
      'всей', 'им', 'наша', 'кажись', 'твоего', 'своей', 'наверное', 'собой', 'нами', 'предположительно', 'мы', 'всех', 'по-моему', 'исстари',
      'себя', 'некто', 'иногда', 'совершенно', 'для', 'вы', 'этих', 'как-то', 'разумеется', 'ему', 'всегда', 'чего', 'другие', 'во-первых',
      'позвольте', 'вип', 'очевидно', 'так-то', 'замыслу', 'нарочно', 'короче', 'то', 'помилуйте', 'сути', 'по-хорошему', 'иначе', 'нередко',
      'чтобы', 'всего', 'все', 'надо', 'весь', 'раз', 'известно', 'из', 'свой', 'по-видимому', 'тут', 'под', 'мой', 'ул', 'луше', 'ей', 'только',
      'вероятно', 'видимо', 'руб', 'слову', 'нет', 'за', 'тем', 'во', 'например', 'помимо', 'такой', 'будет', 'помилуй', 'тоже', 'скажут',
      'видно', 'пожалуй', 'общеизвестно', 'чье', 'вдобавок', 'впрочем', 'соответственно', 'зачастую', 'сам', 'том', 'там', 'полагается',
      'меня', 'однако', 'слышь', 'итак', 'подобное', 'действительно', 'еще', 'его', 'наше', 'данный', 'твой', 'ай', 'необходимо', 'три',
      'далее', 'тот', 'эм', 'же', 'жаль', 'другими', 'видишь', 'некоторых', 'прежде', 'который', 'допустим', 'положено', 'по-нашему', 'весьма',
      'не', 'ро', 'той', 'этого', 'эти', 'напомним', 'правда', 'выходит', 'перед', 'определенно', 'какая', 'относительно', 'сразу', 'имя',
      'чей', 'ее', 'отом', 'неё', 'это', 'два', 'эту', 'разве', 'вне', 'тобы', 'знамо', 'всем', 'вернее', 'будто', 'вновь', 'которые', 'данным',
      'кроме', 'таким', 'нечто', 'кого', 'больше', 'ом', 'уже', 'прочего', 'очень', 'при', 'те', 'вообще', 'ясно', 'ну', 'октябрь', 'до',
      'на', 'в-третьих', 'ежели', 'общем', 'ещё', 'похоже', 'ибо', 'оно', 'ты', 'сверх', 'значит', 'безусловно', 'правильнее', 'ниего', 'него',      'совсем',
      'вами', 'общем-то', 'всю', 'наш', 'всему', 'всякий', 'ой', 'этой', 'ни', 'со', 'вот', 'день', 'может', 'мб', 'тебя', 'нас',
      'оять', 'хотя', 'прочим', 'этим', 'между', 'ним', 'вероятнее', 'бывало', 'примерно', 'вдруг', 'либо']